(a) Data Preprocessing

In [7]:
# Step 1: Install TensorFlow and necessary libraries
!pip install tensorflow numpy

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Example data (small English to French pairs)
data = [("hello", "bonjour"),
        ("how are you", "comment ça va"),
        ("I am fine", "je vais bien"),
        ("what is your name", "comment tu t'appelles"),
        ("my name is", "je m'appelle"),
        ("thank you", "merci"),
        ("goodbye", "au revoir")]

# Splitting the data into input (English) and output (French) pairs
english_sentences, french_sentences = zip(*data)

# Tokenize English sentences
eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(english_sentences)
eng_sequences = eng_tokenizer.texts_to_sequences(english_sentences)
eng_word_index = eng_tokenizer.word_index
max_eng_len = max(len(seq) for seq in eng_sequences)

# Tokenize French sentences
fr_tokenizer = Tokenizer()
fr_tokenizer.fit_on_texts(french_sentences)
fr_sequences = fr_tokenizer.texts_to_sequences(french_sentences)
fr_word_index = fr_tokenizer.word_index
max_fr_len = max(len(seq) for seq in fr_sequences)

# Padding sequences
eng_padded = pad_sequences(eng_sequences, maxlen=max_eng_len, padding='post')
fr_padded = pad_sequences(fr_sequences, maxlen=max_fr_len, padding='post')

# Vocabulary sizes
eng_vocab_size = len(eng_word_index) + 1
fr_vocab_size = len(fr_word_index) + 1


(b) Build Seq2Seq Model

In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

# Encoder
latent_dim = 256

encoder_inputs = Input(shape=(max_eng_len,))
enc_emb = Embedding(eng_vocab_size, latent_dim)(encoder_inputs)

encoder_lstm = LSTM(latent_dim, return_state=True)
_, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_fr_len,))
dec_emb_layer = Embedding(fr_vocab_size, latent_dim)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)

decoder_dense = Dense(fr_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Seq2Seq model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 4)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 3)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 4, 256)         │          3,840 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, 3, 256)         │          3,584 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ [(None, 256), (None,   │        525,312 │ embedding[0][0]        │
│                           │ 256), (None, 256)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ [(None, 3, 256),       │        525,312 │ embedding_1[0][0],     │
│                           │ (None, 256), (None,    │                │ lstm[0][1], lstm[0][2] │
│                           │ 256)]                  │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 3, 14)          │          3,598 │ lstm_1[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,061,646 (4.05 MB)

 Trainable params: 1,061,646 (4.05 MB)

 Non-trainable params: 0 (0.00 B)

(c) Preparing the Data for Training

In [9]:
# Preparing decoder target data (shifted French sentences)
fr_padded_target = np.zeros_like(fr_padded)
fr_padded_target[:, :-1] = fr_padded[:, 1:]

# Train/Val Split
train_size = int(0.8 * len(eng_padded))
eng_train, eng_val = eng_padded[:train_size], eng_padded[train_size:]
fr_train, fr_val = fr_padded[:train_size], fr_padded[train_size:]
fr_target_train, fr_target_val = fr_padded_target[:train_size], fr_padded_target[train_size:]

(d) Train the Model on the Dataset

In [10]:
batch_size = 64
epochs = 100

history = model.fit([eng_train, fr_train], fr_target_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=([eng_val, fr_val], fr_target_val))


Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.0000e+00 - loss: 2.6427 - val_accuracy: 0.8333 - val_loss: 2.5734
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 0.5333 - loss: 2.5904 - val_accuracy: 0.8333 - val_loss: 2.4980
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 370ms/step - accuracy: 0.5333 - loss: 2.5348 - val_accuracy: 0.8333 - val_loss: 2.4089
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step - accuracy: 0.5333 - loss: 2.4702 - val_accuracy: 0.8333 - val_loss: 2.2965
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step - accuracy: 0.5333 - loss: 2.3903 - val_accuracy: 0.8333 - val_loss: 2.1505
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 0.5333 - loss: 2.2882 - val_accuracy: 0.8333 - val_loss: 1.9592
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 0.5333 - loss: 2.1565 - val_accuracy: 0.8333 - val_loss: 1.7128
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.5333 - loss: 1.9894 - val_accuracy: 0.8333 - val

(e) Inference Setup for Translation

In [11]:
# Inference Encoder
encoder_model = Model(encoder_inputs, encoder_states)

# Inference Decoder
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = dec_emb_layer(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(
    dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs2] + decoder_states2)


(f) Translate New Sentences

In [12]:
# Add 'start' and 'end' tokens to French sentences
french_sentences_with_tokens = ['start ' + sent + ' end' for sent in french_sentences]

# Tokenize French sentences with start and end tokens
fr_tokenizer = Tokenizer()
fr_tokenizer.fit_on_texts(french_sentences_with_tokens)
fr_sequences = fr_tokenizer.texts_to_sequences(french_sentences_with_tokens)
fr_word_index = fr_tokenizer.word_index
max_fr_len = max(len(seq) for seq in fr_sequences)

# Padding sequences
eng_padded = pad_sequences(eng_sequences, maxlen=max_eng_len, padding='post')
fr_padded = pad_sequences(fr_sequences, maxlen=max_fr_len, padding='post')


In [2]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    # Start with the 'start' token
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = fr_word_index['start']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = None
        for word, index in fr_word_index.items():
            if index == sampled_token_index:
                sampled_word = word
                break

        if sampled_word == 'end' or len(decoded_sentence) > max_fr_len:
            stop_condition = True
        else:
            decoded_sentence += sampled_word + ' '

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence


In [1]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    # Start with the 'start' token
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = fr_word_index['start']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Get the index of the most likely next word
        sampled_token_index = np.argmax(output_tokens[0, -1, :])

        # Retrieve the corresponding word for the token index
        sampled_word = None
        for word, index in fr_word_index.items():
            if index == sampled_token_index:
                sampled_word = word
                break

        # Handle the case where the word is not found
        if sampled_word is None:
            print(f"Warning: No word found for token index {sampled_token_index}")
            stop_condition = True
        elif sampled_word == 'end' or len(decoded_sentence.split()) > max_fr_len:
            stop_condition = True
        else:
            decoded_sentence += sampled_word + ' '

        # Update the target sequence to the predicted word
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states for the next iteration
        states_value = [h, c]

    return decoded_sentence


In [16]:
def decode_sequence(input_seq):
    # Initial states for the decoder
    states_value = encoder_model.predict(input_seq)

    # Start the sequence with the "start" token
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_token_index['<start>']  # Adjust as per your start token index

    # Loop to generate the translation
    decoded_sentence = ''
    stop_condition = False
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_target_word_index.get(sampled_token_index, '')  # Use '' if None

        # Debugging output for None cases
        if sampled_word is None:
            print("Encountered None for sampled_word, replacing with empty string.")
            sampled_word = ''

        # Append the sampled word to the decoded sentence
        decoded_sentence += sampled_word + ' '

        # Exit condition: either hit max length or the end token
        if sampled_word == '<end>' or len(decoded_sentence.split()) > max_eng_len:
            stop_condition = True

        # Update target sequence and states
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence.strip()  # Remove any trailing spaces


(g) Experimenting and Improving the Model with a Larger Dataset


In [17]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.optimizers import Adam

# Define the latent_dim and other hyperparameters
latent_dim = 256
eng_vocab_size = 15  # Example value; replace with actual vocab size
fr_vocab_size = 15   # Example value; replace with actual vocab size
embedding_dim = 128  # Size of the embedding vectors

# Define encoder
encoder_inputs = Input(shape=(None,))  # Only the sequence length is needed here
encoder_embedding = Embedding(input_dim=eng_vocab_size, output_dim=embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_sequences=True)(encoder_embedding)
encoder_outputs, state_h, state_c = LSTM(latent_dim, return_state=True)(encoder_lstm)
encoder_states = [state_h, state_c]

# Define decoder
decoder_inputs = Input(shape=(None,))  # Only the sequence length is needed here
decoder_embedding = Embedding(input_dim=fr_vocab_size, output_dim=embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True)(decoder_embedding, initial_state=encoder_states)
decoder_outputs = Dense(fr_vocab_size, activation='softmax')(decoder_lstm)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
opt = Adam(learning_rate=0.0005)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Check the shapes of the training data
# Ensure eng_train, fr_train, and fr_target_train have the appropriate shapes
print(f"eng_train shape: {eng_train.shape}, fr_train shape: {fr_train.shape}, fr_target_train shape: {fr_target_train.shape}")

# Train the model
model.fit([eng_train, fr_train], fr_target_train, batch_size=batch_size, epochs=200, validation_data=([eng_val, fr_val], fr_target_val))


eng_train shape: (5, 4), fr_train shape: (5, 3), fr_target_train shape: (5, 3)
Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.0000e+00 - loss: 2.7104 - val_accuracy: 0.6667 - val_loss: 2.6948
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - accuracy: 0.4000 - loss: 2.6972 - val_accuracy: 0.8333 - val_loss: 2.6786
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step - accuracy: 0.5333 - loss: 2.6838 - val_accuracy: 0.8333 - val_loss: 2.6613
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - accuracy: 0.5333 - loss: 2.6696 - val_accuracy: 0.8333 - val_loss: 2.6421
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.5333 - loss: 2.6540 - val_accuracy: 0.8333 - val_loss: 2.6201
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - accuracy: 0.5333 - loss: 2.6364 - val_accuracy: 0.8333 - val_loss: 2.5945
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - accuracy: 0.5333 - loss: 2.6161 - val_accuracy: 0.8333 - val_loss: 2.5641
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━